# Exercise 1.6_Intro to NLP and Network Analysis
In this Notebook we can done the following
1. Import all the required libraries.
2. Load the Spacy english modules to NER.
3. Open the events.txt file that has the Wiki content of the 20th century key events.
4. Apply the NER processing to the txt file and display the same to see the entities.
5. Open the (countries_list_20th_century_1.7) csv file having the standard names of the countries to be compared with the events.txt file. Trim the unwanted spaces from the countries names and create a list with the names.
6. Create a new dictionary named country_variants with the list of the countries that are mentioned with different names in the text.
7. Compare the names in the dictionary and the csv file with the GPE and LOC entities in the NER file generated and count the number of occurence in the txt file.
8. The matching country names and their frequency of occurence in the text file is counted and stored in country_mentioned csv file.
9. The unmatched country names are stored in unmatched_entities files, so that we can compare them and edit the dictionary accordingly to include those names.
10. We have filtered the entities from the text file with the country names, found the relationship between the countries and saved the same in the csv file named 'Relationship_count_NER'.
11. The following the files used and generated
    1. events.txt - Wiki content
    2. countries_list_20th_century_1.7  - List of the countries standard names.
    3. ner_output - NER output converted to dataframe and saved as csv file
    4. country_mentions  - List of countries mentioned in the text file
    5. unmatched_entities - List of entities in the text fil that doesnt have a match in the countries list and dictionary created.
    6. Relationship_count_NER   - List of the relationship between the countries and the count.

In [83]:
# Import the required libraries

import pandas as pd 
import numpy as np
import spacy
from spacy import displacy
import networkx as nx
import os
import matplotlib.pyplot as plt
import scipy
import re
from collections import Counter

!pip install openpyxl

In [84]:
# Executed this command to solve the above error
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.4.1/en_core_web_sm-3.4.1-py3-none-any.whl

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     --------------- ------------------------ 5.0/12.8 MB 30.2 MB/s eta 0:00:01
     ---------------------------------- ---- 11.3/12.8 MB 29.4 MB/s eta 0:00:01
     ---------------------------------------- 12.8/12.8 MB 25.1 MB/s  0:00:00


In [85]:
# Load spacy English module

NER = spacy.load("en_core_web_sm")

In [86]:
# Load the events.txt file that has the content of the 20th century key events scrapped from wiki site.
# Apply the NER to the content

with open('C:/Users/sorna/Desktop/Data Analysis/20th-Century/events.txt', 'r', errors='ignore') as file: 
   data = file.read().replace( '\n', ' ')

book = NER(data)

In [87]:
displacy.render(book[273:20000], style= "ent", jupyter = True)

In [88]:
# Convert the NER file to dataframe and save the same.

entities = [(ent.text, ent.label_) for ent in book.ents]

df_NER = pd.DataFrame(entities, columns=['Entity', 'Label'])
df_NER.to_csv('C:/Users/sorna/Desktop/Data Analysis/20th-Century/ner_output.csv', index=False)


In [89]:
df_NER

Entity     Label
0                   the 20th century      DATE
1               Navigation \t   Main       ORG
2                         Contribute       ORG
3                          HelpLearn       ORG
4            Log in         Personal       ORG
...                              ...       ...
3186  the Wikimedia Foundation, Inc.       ORG
3187               Statistics Cookie       ORG
3188                          Mobile       GPE
3189                the 20th century      DATE
3190                               2  CARDINAL

[3191 rows x 2 columns]

In [90]:
# Read the csv file with the list of countries

df_country = pd.read_csv("C:/Users/sorna/Desktop/Data Analysis/20th-Century/countries_list_20th_century_1.7.csv",index_col=False)
df_country['country_name'] = df_country['country_name'].str.strip()
countries = df_country['country_name'].tolist()

In [91]:
df_country

country_name
0       Afghanistan
1           Albania
2           Algeria
3           Andorra
4            Angola
..              ...
209            USSR
210    North Africa
211          Africa
212  czechoslovakia
213      yugoslavia

[214 rows x 1 columns]

In [92]:
# Create a dictionary with the names from the text file and match them with the standard names in the csv file
# alias -> standard country name

country_variants = {
    'USA': 'United States',
    'US': 'United States',
    'The United States': 'United States',
    'U.S.A' : 'United States',
    'U.S.' : 'United States',
    'America' : 'United States',
    'The Soviet Union' : 'USSR',
    'Soviet Union' : 'USSR',
    'Soviet' : 'USSR',
    'North Korea' : 'Korea,North',
    'South Korea' :'Korea,South',
    'Persia' : 'Iran',
    'Burma' : 'Myanmar',
    'North Vietnam' : 'Vietnam',
    'South Vietnam' : 'Vietnam',
    'Britain' : 'United Kingdom',
    'Great Britain' : 'United Kingdom',
    'London' : 'United Kingdom',
    'North Africa' : 'North Africa',
    'China' : "China, People's Republic of",
}

In [93]:
# Convert everything to lowercase for easy, case-insensitive comparison
variant_to_standard = {k.lower(): v.lower() for k, v in country_variants.items()}

# Include standard countries themselves so they're also recognized
for c in countries:
    variant_to_standard[c.lower()] = c.lower()

In [94]:
# Keep only geographic entities (optional — depends on your NER labels)
NER_countries = df_NER[df_NER['Label'].isin(['GPE', 'LOC'])]

In [95]:
NER_countries

Entity Label
14              Europe   LOC
41    the Panama Canal   LOC
43              Africa   LOC
50            Sarajevo   GPE
59        Young Bosnia   GPE
...                ...   ...
3108           Kagaayi   GPE
3109            Joseph   GPE
3110          Serwadda   GPE
3114            Africa   LOC
3188            Mobile   GPE

[589 rows x 2 columns]

In [113]:
# Compare the LOC and GPE names with the dictionary and the standard country names in the csv file and count its frequency in the text file.

normalized = []
unmatched = []

for ent in NER_countries['Entity']:
    ent_clean = ent.strip().lower()
    if ent_clean in variant_to_standard:
        normalized.append(variant_to_standard[ent_clean])
    else:
        unmatched.append(ent_clean)                                 # save for later review

# Count the Occurrence
counts = Counter(normalized)
freq_df = pd.DataFrame(list(counts.items()), columns=['Country', 'Count'])

#Export the results to the local drive
freq_df.to_csv("C:/Users/sorna/Desktop/Data Analysis/20th-Century/country_mentions.csv", index=False)


# Save the unmatched countries for future reviews
if unmatched:
    unmatched_df = pd.Series(unmatched).value_counts().reset_index()
    unmatched_df.columns = ['Unmatched Entity', 'Frequency']
    unmatched_df.to_csv("C:/Users/sorna/Desktop/Data Analysis/20th-Century/unmatched_entities.csv", index=False)
    

In [114]:
counts

Counter({'united states': 46,
         'ussr': 35,
         'germany': 32,
         'japan': 28,
         'united kingdom': 18,
         'africa': 14,
         'france': 12,
         'italy': 10,
         'poland': 10,
         'india': 9,
         'russia': 8,
         "china, people's republic of": 7,
         'yugoslavia': 5,
         'czechoslovakia': 5,
         'vietnam': 5,
         'finland': 4,
         'norway': 4,
         'north africa': 4,
         'iran': 4,
         'philippines': 4,
         'israel': 4,
         'korea,north': 4,
         'greece': 3,
         'libya': 3,
         'pakistan': 3,
         'romania': 3,
         'cuba': 3,
         'estonia': 2,
         'latvia': 2,
         'lithuania': 2,
         'belgium': 2,
         'albania': 2,
         'egypt': 2,
         'singapore': 2,
         'myanmar': 2,
         'solomon islands': 2,
         'south africa': 2,
         'cambodia': 2,
         'korea,south': 2,
         'austria': 1,
         'spain': 1

In [115]:
freq_df

Country  Count
0        africa     14
1        france     12
2        russia      8
3       germany     32
4    yugoslavia      5
..          ...    ...
60    australia      1
61      vietnam      5
62       rwanda      1
63  afghanistan      1
64      lebanon      1

[65 rows x 2 columns]

In [116]:
unmatched_df

Unmatched Entity  Frequency
0             europe         14
1             berlin         10
2           atlantic          9
3           new york          7
4             allies          7
..               ...        ...
160       buchenwald          1
161          potsdam          1
162           borneo          1
163         iwo jima          1
164           mobile          1

[165 rows x 2 columns]

In [117]:
# This code is used to get the list of the entities present in each sentence.

df_sentences = []

# Loop through sentences, get entity list for each sentence
for sent in book.sents:
       entity_list = [ent.text for ent in sent.ents]
       df_sentences.append({"sentence": sent, "entities": entity_list})

df_sentences = pd.DataFrame(df_sentences)
df_sentences

sentence  \
0                                                (    )   
1     (Key, events, of, the, 20th, century, -, Wikip...   
2     (Search,            , Search,                 ...   
3     (account,  , Log, in,         , Personal, tool...   
4     (", The, war, to, end, all, wars, ", :, World,...   
...                                                 ...   
1576  (This, page, was, last, edited, on, 22, Septem...   
1577  (Text, is, available, under, the, Creative, Co...   
1578  (By, using, this, site, ,, you, agree, to, the...   
1579  (WikipediaÂ, ®, is, a, registered, trademark, ...   
1580  (Privacy, policy, About, Wikipedia, Disclaimer...   

                                               entities  
0                                                    []  
1     [the 20th century, Navigation \t   Main, Contr...  
2                                                    []  
3     [Log in         Personal, the 20th century, th...  
4                           [World War I, Spanish, 1.3]  
...                                                 ...  
1576                   [22 September 2025, 07:30Â, UTC]  
1577  [the Creative Commons Attribution-ShareAlike 4...  
1578              [the Terms of Use and Privacy Policy]  
1579      [WikipediaÂ®, the Wikimedia Foundation, Inc.]  
1580   [Statistics Cookie, Mobile, the 20th century, 2]  

[1581 rows x 2 columns]

In [118]:
# Function to filter out the entities not of interest

countries_lower = {c.lower() for c in countries}

def filter_entity(ent_list, countries_lower, country_variants):
    return [
        ent for ent in ent_list
        if ent.lower() in countries_lower or ent.lower() in country_variants
    ]


In [119]:
df_sentences['country_entities'] = df_sentences['entities'].apply(lambda x: filter_entity(x, countries_lower, country_variants))
df_sentences.head(40)

sentence  \
0                                              (    )   
1   (Key, events, of, the, 20th, century, -, Wikip...   
2   (Search,            , Search,                 ...   
3   (account,  , Log, in,         , Personal, tool...   
4   (", The, war, to, end, all, wars, ", :, World,...   
5      (1.3.1, Economic, depression,         , 1.3.2)   
6   (The, rise, of, dictatorship,           , 1.4,...   
7   (The, war, in, Europe,         , 1.4.2, Blitzk...   
8   (Operation, Overlord,         , 1.4.6, Final, ...   
9   (Allied, offensive,         , 1.4.10, Final, d...   
10                 (The, Holocaust,         , 1.4.12)   
11       (The, Nuclear, Age, begins,           , 1.5)   
12          (The, post, -, war, world,       , 1.5.1)   
13  (The, end, of, empires, :, decolonization,    ...   
14  (The, Cold, War, (, 1947â€“1991, ),         , ...   
15                (The, space, race,         , 1.5.5)   
16  (The, end, of, the, Cold, War,         , 1.5.6...   
17  (English,                  , ReadEditView, his...   
18  (The, World, Wars, sparked, tension, between, ...   
19  (These, advancements, have, played, a, signifi...   
20  (Historic, events, in, the, 20th, century[edit...   
21  (Edwardian, era, The, new, beginning, of, the,...   
22  (The, 1900s, saw, the, decade, herald, a, seri...   
23  (1914, saw, the, completion, of, the, Panama, ...   
24  (The, Scramble, for, Africa, continued, in, th...   
25  (The, atrocities, in, the, Congo, Free, State,...   
26  (From, 1914, to, 1918, ,, the, First, World, W...   
27  (", The, war, to, end, all, wars, ", :, World,...   
28  (World, War, I, Arrest, of, a, suspect, in, Sa...   
29  (Franz, Ferdinand, The, First, World, War, (, ...   
30  (The, war, was, precipitated, by, the, Assassi...   
31  (After, a, period, of, diplomatic, and, milita...   
32  (In, 1917, ,, Russia, ended, hostile, actions,...   
33  (The, Bolsheviks, negotiated, the, Treaty, of,...   
34  (In, the, treaty, ,, Bolshevik, Russia, ceded,...   
35  (It, also, recognized, the, independence, of, ...   
36  (The, war, itself, was, also, a, chance, for, ...   
37  (The, Germans, introduced, the, machine, gun,[...   
38  (Both, sides, had, a, chance, to, test, out, t...   
39  (It, was, widely, believed, that, the, war, wo...   

                                             entities   country_entities  
0                                                  []                 []  
1   [the 20th century, Navigation \t   Main, Contr...                 []  
2                                                  []                 []  
3   [Log in         Personal, the 20th century, th...                 []  
4                         [World War I, Spanish, 1.3]                 []  
5                                                  []                 []  
6              [1.4 Global war:, World War II, 1.4.1]                 []  
7                                     [Europe, 1.4.2]                 []  
8                                                  []                 []  
9                                            [Allied]                 []  
10                                                 []                 []  
11                                              [1.5]                 []  
12                                                 []                 []  
13                                                 []                 []  
14                                     [The Cold War]                 []  
15                                                 []                 []  
16  [the Cold War         1.5.6 Information, 1.6, ...                 []  
17  [pageGet, URLDownload, Download, Wikipedia, Th...                 []  
18                     [the Cold War, the Space Race]                 []  
19                          [the 21st century, today]                 []  
20                     [the 20th century[edit] World]                 []  
21                      [Edwardian, the 20th century]    

In [120]:
# Filter out sentences that don’t have any character entities

df_sentences_filtered = df_sentences[df_sentences['country_entities'].map(len) > 0]

df_sentences_filtered.tail(10)

sentence  \
1143  (", Are, colonial, -, era, borders, drawn, by,...   
1146                 (^, ", 7, Civil, Wars, in, Africa)   
1148                              (This, Is, Africa, .)   
1174  (The, Moldovans, :, Romania, ,, Russia, ,, and...   
1230  (", Selling, ', Operation, Passage, to, Freedo...   
1259  (", Stuck, in, Endless, Preliminaries, :, Viet...   
1507  (", Anti, -, American, Behavior, in, the, Midd...   
1512  (The, Rise, of, China, and, India, :, A, New, ...   
1513               (Singapore, :, World, Scientific, .)   
1539  (", The, History, of, the, HIV, /, AIDS, Epide...   

                                               entities   country_entities  
1143                                [Europeans, Africa]           [Africa]  
1146                             [7 Civil Wars, Africa]           [Africa]  
1148                                           [Africa]           [Africa]  
1174  [Moldovans, Romania, Russia, the Politics of C...  [Romania, Russia]  
1230  [Thomas Dooley, the Religious Overtones of Ear...          [Vietnam]  
1259  [Vietnam, the Battle of the Paris Peace Table,...          [Vietnam]  
1507  [Anti-American, the Middle East, a Field Exper...          [Lebanon]  
1512                         [The Rise of China, India]            [India]  
1513                                        [Singapore]        [Singapore]  
1539     [The History of the HIV/AIDS Epidemic, Africa]           [Africa]

In [121]:
# Defining relationships

# window size = 5 : this defines how many sentences will be looked at simultaneously
relationships = [] # create an empty list

for i in range(df_sentences_filtered.index[-1]):
     end_i = min(i+5, df_sentences_filtered.index[-1])
     country_list = sum((df_sentences_filtered.loc[i: end_i].country_entities), [])

     # Remove duplicated characters that are next to each other
     country_unique = [
         country_list[i] for i in range(len(country_list))
         if (i==0) or country_list[i] != country_list[i-1]
     ]

     if len(country_unique) > 1:
          for idx, a in enumerate( country_unique[:-1]):
              b = country_unique[idx + 1]
              relationships.append({"source": a, "target": b})

In [122]:
relationship_df = pd.DataFrame(relationships)

relationship_df

source     target
0     France     Russia
1     France     Russia
2     Russia    Germany
3    Germany     Russia
4     France     Russia
..       ...        ...
746    India  Singapore
747    India  Singapore
748    India  Singapore
749    India  Singapore
750    India  Singapore

[751 rows x 2 columns]

In [123]:
# Sort the cases with a- >b and b- >a
relationship_df = pd.DataFrame(np.sort(relationship_df.values, axis = 1), columns = relationship_df.columns)
relationship_df.head(5)

source  target
0   France  Russia
1   France  Russia
2  Germany  Russia
3  Germany  Russia
4   France  Russia

In [124]:
relationship_df["value"] = 1
relationship_df = relationship_df.groupby(["source","target"], sort=False, as_index=False).sum()

In [125]:
relationship_df.sort_values(by=['value'],ascending=False).head(40)

source           target  value
10          France          Germany     24
39         Germany            Japan     23
4          Germany            Italy     21
1          Germany           Russia     21
11         Germany           Poland     21
56           India         Pakistan     18
51           Japan  Solomon Islands     16
37           Egypt            Libya     15
40           Japan           Russia     13
15         Estonia           Latvia     12
29          Greece       Yugoslavia     12
31         Albania       Yugoslavia     12
54           India            Japan     12
3   Czechoslovakia       Yugoslavia     12
58          Israel     South Africa     12
20         Denmark           Norway     12
18         Finland          Germany     11
0           France           Russia     11
17         Finland        Lithuania     11
9           France           Poland     11
5          Austria          Germany     11
89            USSR    United States      9
22         Germany           Sweden      6
60            Iran            Libya      6
88            Cuba             USSR      6
87            Cuba         Mongolia      6
52     Philippines  Solomon Islands      6
53           Japan      Philippines      6
85  Czechoslovakia          Hungary      6
55          France            India      6
16          Latvia        Lithuania      6
57          Israel         Pakistan      6
84  Czechoslovakia          Romania      6
83          Poland          Romania      6
82        Bulgaria           Poland      6
47           Japan         Thailand      6
81         Albania         Bulgaria      6
79         Finland          Romania      6
65          Africa            India      6
66        Pakistan      Philippines      6

In [126]:
relationship_df.to_csv('C:/Users/sorna/Desktop/Data Analysis/20th-Century/Relationship_count_NER.csv', index= False)